In [137]:
import numpy as np
import sympy
import csv
from json import loads

lab_number = '2.1.2'


def toCopyString(matrix) -> str:
    return 'matrix(' + str(matrix).replace('Matrix([', '[').replace('])', ']')[1:-1].replace(' ', '') + ')'

def fromCopyString(string: str):
    return loads(string.replace('matrix(', '[').replace(')', ']'))

def получить_матрицы_варианта(number):
    filename = f'./{lab_number}_variants.txt'
    with open(filename, 'r') as file:
        variants = csv.reader(file, delimiter=',')
        variant = list(variants)[number - 1][1].split(';')
        A = fromCopyString(variant[0])
        B = fromCopyString(variant[1])
        L = fromCopyString(variant[2])
        H = fromCopyString(variant[3])
    return A, B, L, H

In [200]:
A, B, L, H = получить_матрицы_варианта(10)
A = sympy.Matrix(A)
B = sympy.Matrix(B)
L = sympy.Matrix(L)
H = sympy.Matrix(H)

In [201]:
def матрица_управляемости(A, B, L):
    матрица = sympy.zeros(3, 6)
    матрица[:, 0:2] = B 
    матрица[:, 2:4] = A*B
    матрица[:, 4:] = A**2*B
    return матрица

In [202]:
S = матрица_управляемости(A, B, L)
S

Matrix([
[-5,  9,  14, -63, -104,  414],
[ 4,  9, -17,   9,  -23,  -18],
[-2, -9, -16,  54,   88, -360]])

In [203]:
toCopyString(S)

'matrix([-5,9,14,-63,-104,414],[4,9,-17,9,-23,-18],[-2,-9,-16,54,88,-360])'

In [182]:
def определитель_матрицы_управляемости(S):
    определитель = S[:, :3].det()
    # print('Да!' if определитель != 0 else 'Нет :(')
    return определитель

In [204]:
print(определитель_матрицы_управляемости(S))

2115


In [144]:
def коэффициенты_характеристического_полином_разомкнутой_системы(A):
    l = sympy.symbols('lambda')
    I = sympy.eye(3)
    detChPolyOpen = (l*I - A).det()
    c0 = detChPolyOpen.coeff(l, 0)
    c1 = detChPolyOpen.coeff(l, 1)
    c2 = detChPolyOpen.coeff(l, 2)
    ChPolyOpen = sympy.Matrix([
        c0,
        c1,
        c2
        ])
    return ChPolyOpen.T

In [205]:
c = коэффициенты_характеристического_полином_разомкнутой_системы(A)
c

Matrix([[4, -10, 5]])

In [206]:
toCopyString(c)

'matrix([4,-10,5])'

In [148]:
def коэффициенты_характеристического_полином_замкнутой_системы(L):
    c0 = L[0]*L[1]*L[2]
    c1 = L[0]*L[1] + L[1]*L[2] + L[0]*L[2]
    c2 = L[0]+L[1]+L[2]

    ChPolyOpen = sympy.Matrix([
        -c0,
        c1,
        -c2
        ])
    return ChPolyOpen.T

In [207]:
coe = коэффициенты_характеристического_полином_замкнутой_системы(L)
coe

Matrix([[90, 63, 14]])

In [208]:
toCopyString(coe)

'matrix([90,63,14])'

In [150]:
def вектор_из_коэффициентов(A, B, L, H):
    return коэффициенты_характеристического_полином_разомкнутой_системы(A) - коэффициенты_характеристического_полином_замкнутой_системы(L)

In [236]:
def вектор_r(A, B, L, H):
    r1 = sympy.Symbol('r_1')
    r = sympy.Matrix([r1, 1])
    b = B*r

    c0, c1, c2 = коэффициенты_характеристического_полином_разомкнутой_системы(A)
    d3 = b 
    d2 = A*d3 + c2*b
    d1 = A*d2 + c1*b

    D = sympy.eye(3)
    D[:, 0] = d1
    D[:, 1] = d2
    D[:, 2] = d3
    m = D.T*H.T

    n1 = 1
    n2 = sympy.Symbol('n_2')
    n3 = sympy.Symbol('n_3')

    n = sympy.Matrix([n1, n2, n3]).T

    solved = sympy.solvers.solvers.solve((n*m).T, Msymbols=[n2, n3])
    n_2 = solved[0][n2]
    n_3 = solved[0][n3]
    denominator = sympy.fraction(n_2)[1]

    N = sympy.Matrix([n1*denominator, n_2*denominator, n_3*denominator])

    vec = вектор_из_коэффициентов(A, B, L, H)
    res = (vec*N)[0]
    solved_ = sympy.solvers.solvers.solve(res, Msymbols=[r1])
    r_1 = solved_[0]
    r = sympy.Matrix([r_1, 1])
    return r.n()

In [237]:
r = вектор_r(A, B, L, H)
r

IndexError: list index out of range

In [191]:
toCopyString(r)

'matrix([-0.537336764877053],[1.00000000000000])'

In [159]:
def матрица_D_H(A, B, L, H, r):
    c0, c1, c2 = коэффициенты_характеристического_полином_разомкнутой_системы(A)
    k0, k1, k2 = коэффициенты_характеристического_полином_замкнутой_системы(L)
    d3 = B*r
    d2 = A*d3 + c2*B*r
    d1 = A*d2 + c1*B*r
    res = sympy.eye(3)
    res[:, 0] = d1
    res[:, 1] = d2
    res[:, 2] = d3
    return (res.T*H.T)

In [192]:
rs = матрица_D_H(A, B, L, H, r)
rs[:2, :].inv()

Matrix([
[0.00240259189265025, -0.195598425223079],
[ 0.0141384146100731,  -0.21915778295871]])

In [193]:
toCopyString(rs)

'matrix([-97.8861500407386,87.3634353324599],[-6.31487942237728,1.07311028299156],[0.224020589262317,-0.402611761967154])'

In [194]:
вектор_из_коэффициентов(A, B, L, H)[:, :2]

Matrix([[-34, -94]])

In [195]:
def вектор_v(A, B, L, H, r):

    res = (матрица_D_H(A, B, L, H, r)[:2, :].inv())*(вектор_из_коэффициентов(A, B, L, H))[:, :2].T

    return res.n().T

In [196]:
res = вектор_v(A, B, L, H, r)
res

Matrix([[18.3045638466193, 20.1201255013762]])

In [197]:
toCopyString(res)

'matrix([18.3045638466193,20.1201255013762])'

In [171]:
def матрица_p(v, r):
    return r*v

In [198]:
RE = матрица_p(res, r)
RE

Matrix([
[-9.8357151198279, -10.8112831458298],
[18.3045638466193,  20.1201255013762]])

In [199]:
toCopyString(RE)

'matrix([-9.83571511982790,-10.8112831458298],[18.3045638466193,20.1201255013762])'

In [239]:
for var_number in range(1, 31):
    A, B, L, H = получить_матрицы_варианта(var_number)
    A = sympy.Matrix(A)
    B = sympy.Matrix(B)
    L = sympy.Matrix(L)
    H = sympy.Matrix(H)
    try:
        S                        = матрица_управляемости(A, B, L)
        определитель             = определитель_матрицы_управляемости(S)
        коэффициенты_разомкнутой = коэффициенты_характеристического_полином_разомкнутой_системы(A)
        коэффициенты_замкнутой   = коэффициенты_характеристического_полином_замкнутой_системы(L)
        r                        = вектор_r(A, B, L, H)
        DH_матрица               = матрица_D_H(A, B, L, H, r)
        v                        = вектор_v(A, B, L, H, r)
        u                        = матрица_p(v, r)

        print(var_number)
        with open(f'./results/{var_number}.txt', 'w') as file:
            file.write(toCopyString(S) + '\n')
            file.write(str(определитель) + '\n')
            file.write(toCopyString(коэффициенты_разомкнутой) + '\n')
            file.write(toCopyString(коэффициенты_замкнутой) + '\n')
            file.write(toCopyString(r) + '\n')
            file.write(toCopyString(DH_матрица) + '\n')
            file.write(toCopyString(v) + '\n')
            file.write(toCopyString(u) + '\n')
    except:
        pass
        

1
2
3
4
5
6
7
8
9
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
